In [1]:
import pandas as pd
import numpy as np
import torch
from gensim.models import Word2Vec
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

2024-10-28 21:12:17.823649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 21:12:20.593980: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-28 21:12:20.594026: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-28 21:12:20.897401: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 21:12:24.606871: W tensorflow/stream_executor/platform/de

In [2]:
chessData = pd.read_csv('Data/chessData.csv')
random_evals = pd.read_csv('Data/random_evals.csv')
tactic_evals = pd.read_csv('Data/tactic_evals.csv')

In [3]:
tactic_evals_check = tactic_evals.dropna()
tactic_evals_check['isCheck'] = tactic_evals_check['Evaluation'].str.contains("#")
tactic_evals_check['isCheck'] = list(map(int, tactic_evals_check['isCheck']))

predictors_raw = tactic_evals_check.drop(['Evaluation', 'isCheck'], axis=1)
predictors_raw.head()

/tmp/ipykernel_9480/2706941485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tactic_evals_check['isCheck'] = tactic_evals_check['Evaluation'].str.contains("#")
/tmp/ipykernel_9480/2706941485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tactic_evals_check['isCheck'] = list(map(int, tactic_evals_check['isCheck']))


,FEN,Move
0,r2qkbr1/pb1nn3/1ppp3p/8/3P1p2/2PB1N1P/PPQN1PP1...,d3g6
1,r2qkb2/pb1nn3/1ppp2rp/8/3P1p2/2P2N1P/PPQN1PP1/...,c2g6
2,r2qkbr1/pb1nn3/1ppp2Bp/8/3P1p2/2P2N1P/PPQN1PP1...,g8g6
3,8/7p/R5p1/2p1pkP1/7P/P4PK1/1r6/3q4 w - - 6 46,a6f6
4,6k1/pp6/3p4/2p1p3/2P1P1q1/1P1P2pP/P5P1/5K2 w -...,h3g4


In [4]:
def separate_board(board):
    return board.replace('/', ' ').split()

def separate_move(move):
    separated_move = []
    for i in range(len(move)):
        separated_move.append(move[i])
    return separated_move


In [5]:
tokens_lines = list(map(separate_board, np.asarray(tactic_evals_check['FEN'])))
tokens_lines = [line[0:8] for line in tokens_lines]

tokens_moves = list(map(separate_move, np.asarray(tactic_evals_check['Move'])))
#tokens_piece = list(map(fen_to_tokens_without_spaces, np.asarray(tactic_evals['FEN'])))
predictors_raw['FEN'] = tokens_lines
predictors_raw['Move'] = tokens_moves


In [6]:
embedding_dim = 64
fen_embeddings = Word2Vec(sentences=tokens_lines, vector_size=embedding_dim, window=5, min_count=1, sg=1)
fen_embeddings.train(tokens_lines, total_examples=len(tokens_lines), epochs=10)

(173507683, 203556080)

In [7]:
embedding_dim = 64
move_embeddings = Word2Vec(sentences=tokens_moves, vector_size=embedding_dim, window=5, min_count=1, sg=1)
move_embeddings.train(tokens_moves, total_examples=len(tokens_moves), epochs=10)

(14528587, 101898870)

In [ ]:
fen_data = []
move_data = []

for line in range(len(tokens_lines)):
    fen_data.append(fen_embeddings.wv[tokens_lines[line]])

for move in range(len(tokens_moves)):
    move_data.append(move_embeddings.wv[tokens_moves[move]])

predictors = pd.DataFrame({
    'fen_data': fen_data,
    'move_data': move_data
})

In [ ]:
ncols = predictors.shape[1]
mate_evaluator = Sequential()
mate_evaluator.add(Dense(64, activation = 'relu', input_shape = (ncols,)))
mate_evaluator.add(Dense(64, activation = 'relu'))
mate_evaluator.add(Dense(1 , activation = 'sigmoid'))
mate_evaluator.compile(optimizer = 'adam', loss = 'categorical_crossentropy', 
		      metrics=['accuracy'])


In [ ]:
early_stopping_monitor = EarlyStopping(patience=2)
mate_evaluator.fit(predictors, tactic_evals_check['isCheck'], 
                    validation_split=0.3, 
                    epochs=20, 
                    callbacks=[early_stopping_monitor])